In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
drop_cols = ['主键', '婚姻状况', '职业', '职称', '职务', '学历', '单位月缴存额', '是否逾期']
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# drop_cols = ['主键', '是否逾期']

In [2]:
root = '../data'
col_dict = {
    'id': '主键',
    'XINGBIE': '性别',
    'CSNY': '出生年月',
    'HYZK': '婚姻状况',
    'ZHIYE': '职业',
    'ZHICHEN': '职称',
    'ZHIWU': '职务',
    'XUELI': '学历',
    'DWJJLX': '单位经济类型',
    'DWSSHY': '单位所属行业',
    'GRJCJS': '个人缴存基数',
    'GRZHZT': '个人账户状态',
    'GRZHYE': '个人账户余额',
    'GRZHSNJZYE': '个人账户上年结转余额',
    'GRZHDNGJYE': '个人账户当年归集余额', 
    'GRYJCE': '个人月缴存额',
    'DWYJCE': '单位月缴存额',
    'DKFFE': '贷款发放额',
    'DKYE': '贷款余额',
    'DKLL': '贷款利率',
    'label': '是否逾期'
}

In [3]:
# 目标编码
def kfold_mean(df_train, df_test, target, target_mean_list):
    folds = StratifiedKFold(n_splits=5, random_state=42)

    mean_of_target = df_train[target].mean()

    for fold_, (trn_idx, val_idx) in tqdm(
            enumerate(folds.split(df_train, y=df_train[target]))):
        tr_x = df_train.iloc[trn_idx, :]
        vl_x = df_train.iloc[val_idx, :]

        for col in target_mean_list:
            df_train.loc[vl_x.index, f'{col}_target_enc'] = vl_x[col].map(
                tr_x.groupby(col)[target].mean())

    for col in target_mean_list:
        df_train[f'{col}_target_enc'] = df_train[col].map(
            df_train.groupby(col)[f'{col}_target_enc'].mean())
        
        df_test[f'{col}_target_enc'] = df_test[col].map(
            df_train.groupby(col)[f'{col}_target_enc'].mean())

    return pd.concat([df_train, df_test], ignore_index=True)


In [4]:
def tpr_weight_funtion(y_predict,y_true):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]

    return 'tpr', 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3, True

In [5]:
def monthly_house_payments(p, i):
    if i == 2.708 or i == 2.979:
        n = 30*12
    else:
        n = 5*12
    i = i/100
    return p*i*(1+i)**n/((1+i)**n-1)

In [6]:
dt = "2021-1-01 00:00:00"
timestamp_assign = time.strptime(dt, "%Y-%m-%d %H:%M:%S")
timestamp_assign=time.mktime(timestamp_assign)
def convert(x):
    age = (datetime.datetime.fromtimestamp(timestamp_assign)-datetime.datetime.fromtimestamp(x)).days
    return age//365+1

In [7]:
train = pd.read_csv(root+'/train.csv')
test = pd.read_csv(root+'/test.csv')
submit = pd.read_csv(root+'/submit.csv')
train['CSNY'] = train['CSNY'].apply(convert)
test['CSNY'] = test['CSNY'].apply(convert)

In [8]:
train.columns=train.columns.map(col_dict)
test.columns=test.columns.map(col_dict)
for col in [f for f in train.select_dtypes('int64').columns if f not in ['是否逾期', '贷款发放额']]:
    print(col)
    train[col].fillna('-1', inplace=True)
    test[col].fillna('-1', inplace=True)
    le = LabelEncoder()
    le.fit(pd.concat([train[[col]], test[[col]]], axis=0, ignore_index=True))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

性别
出生年月
婚姻状况
职业
职称
职务
学历
单位经济类型
单位所属行业
个人账户状态


In [9]:
#首套
#5年以上3.25： 2.708
#1-5年 2.75： 2.292
#二套
#5年以上 3.575： 2.979
#1-5年 3.025： 2.521
rate_dict = {3.025: 2.521, 3.575: 2.979, 3.25: 2.708, 2.75: 2.292}

def rate_func(x):
    if x == 3.025:
        return 2.521
    if x == 3.575:
        return 2.979
    if x == 3.25:
        return 2.708
    if x == 2.75:
        return 2.292
    return x

def loan_years(x):
    if x == 2.708 or x == 2.979:
        return 0
    return 1

def num_house(x):
    if x == 2.521 or x== 2.979:
        return 0
    return 1

In [10]:
def ability_pay(gz, gjj, yg):
    #还贷能力系数
    return (gz+gjj)/yg

In [11]:
def combine_feat(x, y):
    feat_dic = {}
    for i in range(len(x)):
        pair = (x[i], y[i])
        feat_dic[pair] = feat_dic.get(pair, 0) + 1

    return feat_dic

def HYJJLX(x, y, company_feat):
    #行业+经济类型
    if (x, y) in company_feat:
        return company_feat[(x, y)]
    return -1

company_feat = combine_feat(list(train['单位经济类型']), list(train['单位所属行业']))
company_feat = {k: v for k, v in company_feat.items() if v>50}

In [12]:
train['贷款利率'] = train['贷款利率'].apply(rate_func)
test['贷款利率'] = test['贷款利率'].apply(rate_func)
feature_list =  ['单位经济类型', '单位所属行业', '个人账户状态']
data = pd.concat([train, test], ignore_index=True)
# data = kfold_mean(data[~data['是否逾期'].isna()], data[data['是否逾期'].isna()],
#                   '是否逾期',
#                   feature_list)

# 频数统计

for col in feature_list:
    data[col + '_COUNT'] = data[col].map(data[col].value_counts())
    col_idx = data[col].value_counts()
    for idx in col_idx[col_idx < 5].index:
        data[col] = data[col].replace(idx, -1)  
        
# 偏离值特征
group_list = ['单位经济类型', '单位所属行业', '个人账户状态']
num_feature_list = ['个人月缴存额', '贷款发放额', '贷款余额', '个人缴存基数',
                    '个人账户上年结转余额', '个人账户当年归集余额']                   
for group in group_list:
    for feature in num_feature_list:
        tmp = data.groupby(group)[feature].agg([sum, min, max, np.mean]).reset_index()
        tmp = pd.merge(data, tmp, on=group, how='left')
        data['{}-mean_gb_{}'.format(feature, group)] = data[feature] - tmp['mean']
        data['{}-min_gb_{}'.format(feature, group)] = data[feature] - tmp['min']
        data['{}-max_gb_{}'.format(feature, group)] = data[feature] - tmp['max']
        data['{}/sum_gb_{}'.format(feature, group)] = data[feature] / tmp['sum']  
train, test = data[~data['是否逾期'].isna()], data[data['是否逾期'].isna()]


In [13]:

raw_feat_cols = [col for col in list(train.columns) if col not in drop_cols ]
train_data = train[raw_feat_cols]
test_data = test[raw_feat_cols]
train_label = train['是否逾期']

In [14]:
train_data['行业+经济类型'] = train_data.apply(lambda row: HYJJLX(row['单位经济类型'], row['单位经济类型'], company_feat), axis=1)
train_data['月供'] = train_data.apply(lambda row: monthly_house_payments(row['贷款发放额'], row['贷款利率']), axis=1)
train_data['月供2'] = train_data.apply(lambda row: monthly_house_payments(row['贷款余额'], row['贷款利率']), axis=1)
train_data['还贷能力系数'] = train_data.apply(lambda row: ability_pay(row['个人缴存基数'], row['个人月缴存额'], row['月供']), axis=1)
train_data['月供/个人缴存基数'] = train_data['月供']/train_data['个人缴存基数']
train_data['月供/个人月缴存额'] = train_data['月供']/train_data['个人月缴存额']
train_data['归集余额+结转余额'] = train_data['个人账户上年结转余额']+train_data['个人账户当年归集余额']
train_data['归集余额+结转余额-个人账户余额'] = train_data['归集余额+结转余额']-train_data['个人账户余额']
train_data['个人月缴存额/贷款余额'] = train_data['个人月缴存额']/train_data['贷款余额']
train_data['个人账户余额/贷款余额'] = train_data['个人账户余额']/train_data['贷款余额']
train_data['个人月缴存额/贷款发放额'] = train_data['个人月缴存额']/train_data['贷款发放额']
train_data['贷款利率*贷款发放额'] = train_data['贷款利率']*train_data['贷款发放额']
train_data['贷款利率*贷款余额'] = train_data['贷款利率']*train_data['贷款余额']
train_data['个人缴存基数/个人账户余额'] = train_data['个人缴存基数']*train_data['个人账户余额']
train_data['个人缴存基数/个人账户上年结转余额'] = train_data['个人缴存基数']*train_data['个人账户上年结转余额']
train_data['个人缴存基数/贷款发放额'] = train_data['个人缴存基数']*train_data['贷款发放额']
train_data['个人缴存基数/贷款余额'] = train_data['个人缴存基数']*train_data['贷款余额']
train_data['贷款利率/个人缴存基数'] = train_data['贷款利率']/train_data['个人缴存基数']
train_data['公积金比例'] = train_data['个人月缴存额']/train_data['个人缴存基数']
train_data['贷款年限类别'] = train_data['贷款利率'].apply(loan_years)
train_data['第N房'] = train_data['贷款利率'].apply(num_house)
train_data['已还贷款'] = train_data['贷款发放额']-train_data['贷款余额']
train_data['贷款余额/公积金'] = train_data['贷款余额']/train_data['个人缴存基数']
train_data['已还贷款比例'] = train_data['已还贷款']/train_data['贷款发放额']

test_data['行业+经济类型'] = test_data.apply(lambda row: HYJJLX(row['单位经济类型'], row['单位经济类型'], company_feat), axis=1)
test_data['月供'] = test_data.apply(lambda row: monthly_house_payments(row['贷款发放额'], row['贷款利率']), axis=1)
test_data['月供2'] = test_data.apply(lambda row: monthly_house_payments(row['贷款余额'], row['贷款利率']), axis=1)
test_data['还贷能力系数'] = test_data.apply(lambda row: ability_pay(row['个人缴存基数'], row['个人月缴存额'], row['月供']), axis=1)
test_data['月供/个人缴存基数'] = test_data['月供']/test_data['个人缴存基数']
test_data['月供/个人月缴存额'] = test_data['月供']/test_data['个人月缴存额']
test_data['归集余额+结转余额'] = test_data['个人账户上年结转余额']+test_data['个人账户当年归集余额']
test_data['归集余额+结转余额-个人账户余额'] = test_data['归集余额+结转余额']-test_data['个人账户余额']
test_data['个人月缴存额/贷款余额'] = test_data['个人月缴存额']/test_data['贷款余额']
test_data['个人账户余额/贷款余额'] = test_data['个人账户余额']/test_data['贷款余额']
test_data['个人月缴存额/贷款发放额'] = test_data['个人月缴存额']/test_data['贷款发放额']
test_data['贷款利率*贷款发放额'] = test_data['贷款利率']*test_data['贷款发放额']
test_data['贷款利率*贷款余额'] = test_data['贷款利率']*test_data['贷款余额']
test_data['个人缴存基数/个人账户余额'] = test_data['个人缴存基数']*test_data['个人账户余额']
test_data['个人缴存基数/个人账户上年结转余额'] = test_data['个人缴存基数']*test_data['个人账户上年结转余额']
test_data['个人缴存基数/贷款发放额'] = test_data['个人缴存基数']*test_data['贷款发放额']
test_data['个人缴存基数/贷款余额'] = test_data['个人缴存基数']*test_data['贷款余额']
test_data['贷款利率/个人缴存基数'] = test_data['贷款利率']/test_data['个人缴存基数']
test_data['公积金比例'] = test_data['个人月缴存额']/test_data['个人缴存基数']
test_data['贷款年限类别'] = test_data['贷款利率'].apply(loan_years)
test_data['第N房'] = test_data['贷款利率'].apply(num_house)
test_data['已还贷款'] = test_data['贷款发放额']-test_data['贷款余额']
test_data['贷款余额/公积金'] = test_data['贷款余额']/test_data['个人缴存基数']
test_data['已还贷款比例'] = test_data['已还贷款']/test_data['贷款发放额']

In [15]:
import xgboost as xgb
from xgboost import XGBClassifier

params = {
        'booster':'gbtree',
        'objective': 'binary:logistic',
        'metric': 'auc',
        'num_leaves': 31,
        'max_bin': 50,
#         'max_depth': 6,
        "learning_rate": 0.1,
        "colsample_bytree": 0.8,  # 每次迭代中随机选择特征的比例
        "bagging_fraction": 0.8,  # 每次迭代时用的数据比例
        'min_child_samples': 25,
        'n_jobs': -1,
        'silent': True,  # 信息输出设置成1则没有信息输出
        'seed': 1208,
        'scale_pos_weight':0.1,
#         'lambda_l1':0.3,
#         'lambda_l2':0.2,
#     'min_split_gain':0.2,
        'verbose' : -1
    }  #设置出参数


In [24]:
KF = StratifiedKFold(n_splits=5, random_state=2020)
oof_lgb = np.zeros(len(train_data))
predictions_lgb = np.zeros((len(test_data)))
feat_importance_table = pd.DataFrame(columns=['feat', 'importance'])
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train_data.values, train_label.values)):
    trn_data = xgb.DMatrix(train_data.iloc[trn_idx],train_label.iloc[trn_idx])    
    val_data = xgb.DMatrix(train_data.iloc[val_idx],train_label.iloc[val_idx])

    num_round = 10000
    clf = xgb.train(
                    params,
                    trn_data,
                    num_round,
                    evals = [(val_data,'val')],
                    early_stopping_rounds=1000,  
    )
    
    val_pred = clf.predict(xgb.DMatrix(train_data.iloc[val_idx]), ntree_limit=clf.best_ntree_limit)
    oof_lgb[val_idx] = val_pred
    predictions_lgb[:] += clf.predict(xgb.DMatrix(test_data), ntree_limit=clf.best_ntree_limit) 
    print('--------------------------------'*2)
    print("TPR: {}".format(tpr_weight_funtion(val_pred, train_label.iloc[val_idx])))
    print('--------------------------------'*2)
    feat_importance_table['feat'] = clf.feature_names
    feat_importance_table['importance'+str(fold_)] = feat_importance_table['feat'].map(clf.get_fscore())
feat_importance_table['importance'] = feat_importance_table.mean(axis=1)

print("AUC score: {}".format(roc_auc_score(train_label, oof_lgb)))
print("TPR weight: {}".format(tpr_weight_funtion(oof_lgb, train_label)))

[0]	val-error:0.053493
Will train until val-error hasn't improved in 1000 rounds.
[1]	val-error:0.052868
[2]	val-error:0.052743
[3]	val-error:0.060117
[4]	val-error:0.060117
[5]	val-error:0.060242
[6]	val-error:0.054368
[7]	val-error:0.054243
[8]	val-error:0.052868
[9]	val-error:0.052743
[10]	val-error:0.052743
[11]	val-error:0.052868
[12]	val-error:0.052868
[13]	val-error:0.052743
[14]	val-error:0.052743
[15]	val-error:0.052743
[16]	val-error:0.052743
[17]	val-error:0.052868
[18]	val-error:0.052868
[19]	val-error:0.052993
[20]	val-error:0.052868
[21]	val-error:0.052868
[22]	val-error:0.052743
[23]	val-error:0.052868
[24]	val-error:0.052368
[25]	val-error:0.052243
[26]	val-error:0.051994
[27]	val-error:0.052118
[28]	val-error:0.051744
[29]	val-error:0.051244
[30]	val-error:0.051244
[31]	val-error:0.051244
[32]	val-error:0.050994
[33]	val-error:0.050869
[34]	val-error:0.050869
[35]	val-error:0.050744
[36]	val-error:0.050744
[37]	val-error:0.050869
[38]	val-error:0.050744
[39]	val-error:

[333]	val-error:0.043245
[334]	val-error:0.043245
[335]	val-error:0.043245
[336]	val-error:0.043245
[337]	val-error:0.043245
[338]	val-error:0.043245
[339]	val-error:0.04312
[340]	val-error:0.042995
[341]	val-error:0.04287
[342]	val-error:0.04312
[343]	val-error:0.04312
[344]	val-error:0.04312
[345]	val-error:0.04312
[346]	val-error:0.04312
[347]	val-error:0.04312
[348]	val-error:0.042995
[349]	val-error:0.04312
[350]	val-error:0.04312
[351]	val-error:0.04287
[352]	val-error:0.04287
[353]	val-error:0.04287
[354]	val-error:0.04287
[355]	val-error:0.04287
[356]	val-error:0.04287
[357]	val-error:0.042995
[358]	val-error:0.042995
[359]	val-error:0.042995
[360]	val-error:0.042995
[361]	val-error:0.042995
[362]	val-error:0.04312
[363]	val-error:0.04312
[364]	val-error:0.042995
[365]	val-error:0.042995
[366]	val-error:0.042995
[367]	val-error:0.042995
[368]	val-error:0.04287
[369]	val-error:0.04287
[370]	val-error:0.04287
[371]	val-error:0.04287
[372]	val-error:0.042995
[373]	val-error:0.0429

[668]	val-error:0.04137
[669]	val-error:0.04137
[670]	val-error:0.04137
[671]	val-error:0.04137
[672]	val-error:0.041495
[673]	val-error:0.041495
[674]	val-error:0.041495
[675]	val-error:0.041495
[676]	val-error:0.041495
[677]	val-error:0.041495
[678]	val-error:0.041495
[679]	val-error:0.041495
[680]	val-error:0.04162
[681]	val-error:0.04162
[682]	val-error:0.04162
[683]	val-error:0.04162
[684]	val-error:0.04162
[685]	val-error:0.04162
[686]	val-error:0.041495
[687]	val-error:0.04162
[688]	val-error:0.041745
[689]	val-error:0.041745
[690]	val-error:0.041745
[691]	val-error:0.041745
[692]	val-error:0.041745
[693]	val-error:0.041745
[694]	val-error:0.04162
[695]	val-error:0.04162
[696]	val-error:0.04162
[697]	val-error:0.04162
[698]	val-error:0.04162
[699]	val-error:0.041745
[700]	val-error:0.04162
[701]	val-error:0.04162
[702]	val-error:0.04162
[703]	val-error:0.04162
[704]	val-error:0.04162
[705]	val-error:0.04162
[706]	val-error:0.04162
[707]	val-error:0.04162
[708]	val-error:0.04162


[1004]	val-error:0.04137
[1005]	val-error:0.041245
[1006]	val-error:0.041245
[1007]	val-error:0.041245
[1008]	val-error:0.041245
[1009]	val-error:0.041245
[1010]	val-error:0.041245
[1011]	val-error:0.041245
[1012]	val-error:0.041245
[1013]	val-error:0.04137
[1014]	val-error:0.041245
[1015]	val-error:0.041245
[1016]	val-error:0.041245
[1017]	val-error:0.041245
[1018]	val-error:0.041245
[1019]	val-error:0.04112
[1020]	val-error:0.040995
[1021]	val-error:0.040995
[1022]	val-error:0.040995
[1023]	val-error:0.04087
[1024]	val-error:0.040995
[1025]	val-error:0.040995
[1026]	val-error:0.040995
[1027]	val-error:0.040995
[1028]	val-error:0.04087
[1029]	val-error:0.04087
[1030]	val-error:0.04087
[1031]	val-error:0.04087
[1032]	val-error:0.04087
[1033]	val-error:0.04087
[1034]	val-error:0.04087
[1035]	val-error:0.04087
[1036]	val-error:0.04087
[1037]	val-error:0.040745
[1038]	val-error:0.040745
[1039]	val-error:0.040745
[1040]	val-error:0.040745
[1041]	val-error:0.040745
[1042]	val-error:0.040745

[1325]	val-error:0.04062
[1326]	val-error:0.040745
[1327]	val-error:0.040745
[1328]	val-error:0.040745
[1329]	val-error:0.040745
[1330]	val-error:0.040495
[1331]	val-error:0.040495
[1332]	val-error:0.04062
[1333]	val-error:0.04062
[1334]	val-error:0.04062
[1335]	val-error:0.040745
[1336]	val-error:0.04062
[1337]	val-error:0.04062
[1338]	val-error:0.04062
[1339]	val-error:0.040495
[1340]	val-error:0.040495
[1341]	val-error:0.04062
[1342]	val-error:0.04062
[1343]	val-error:0.04062
[1344]	val-error:0.04062
[1345]	val-error:0.04062
[1346]	val-error:0.04062
[1347]	val-error:0.04087
[1348]	val-error:0.04087
[1349]	val-error:0.040995
[1350]	val-error:0.04087
[1351]	val-error:0.040745
[1352]	val-error:0.04087
[1353]	val-error:0.04087
[1354]	val-error:0.04087
[1355]	val-error:0.040745
[1356]	val-error:0.040745
[1357]	val-error:0.040745
[1358]	val-error:0.040745
[1359]	val-error:0.040745
[1360]	val-error:0.040745
[1361]	val-error:0.040745
[1362]	val-error:0.040745
[1363]	val-error:0.040745
[1364

...

[714]	val-error:0.04213
[715]	val-error:0.042005
[716]	val-error:0.042005
[717]	val-error:0.042005
[718]	val-error:0.042255
[719]	val-error:0.04213
[720]	val-error:0.04213
[721]	val-error:0.04213
[722]	val-error:0.04213
[723]	val-error:0.04213
[724]	val-error:0.04213
[725]	val-error:0.04213
[726]	val-error:0.04213
[727]	val-error:0.04213
[728]	val-error:0.042255
[729]	val-error:0.04213
[730]	val-error:0.04213
[731]	val-error:0.042005
[732]	val-error:0.042005
[733]	val-error:0.042005
[734]	val-error:0.04188
[735]	val-error:0.04213
[736]	val-error:0.04213
[737]	val-error:0.04213
[738]	val-error:0.04213
[739]	val-error:0.04213
[740]	val-error:0.042005
[741]	val-error:0.042005
[742]	val-error:0.042005
[743]	val-error:0.04213
[744]	val-error:0.042005
[745]	val-error:0.042005
[746]	val-error:0.04213
[747]	val-error:0.04213
[748]	val-error:0.042005
[749]	val-error:0.042005
[750]	val-error:0.042005
[751]	val-error:0.042005
[752]	val-error:0.042005
[753]	val-error:0.04188
[754]	val-error:0.0420

[1047]	val-error:0.04138
[1048]	val-error:0.041255
[1049]	val-error:0.04138
[1050]	val-error:0.041505
[1051]	val-error:0.041505
[1052]	val-error:0.04138
[1053]	val-error:0.04138
[1054]	val-error:0.041505
[1055]	val-error:0.041505
[1056]	val-error:0.04138
[1057]	val-error:0.04138
[1058]	val-error:0.04138
[1059]	val-error:0.04138
[1060]	val-error:0.04138
[1061]	val-error:0.04138
[1062]	val-error:0.04138
[1063]	val-error:0.04138
[1064]	val-error:0.04138
[1065]	val-error:0.04138
[1066]	val-error:0.04138
[1067]	val-error:0.04138
[1068]	val-error:0.04138
[1069]	val-error:0.04138
[1070]	val-error:0.04138
[1071]	val-error:0.04138
[1072]	val-error:0.041255
[1073]	val-error:0.041255
[1074]	val-error:0.041255
[1075]	val-error:0.041255
[1076]	val-error:0.041255
[1077]	val-error:0.041255
[1078]	val-error:0.04138
[1079]	val-error:0.04138
[1080]	val-error:0.04138
[1081]	val-error:0.041255
[1082]	val-error:0.041005
[1083]	val-error:0.041005
[1084]	val-error:0.04113
[1085]	val-error:0.04113
[1086]	val-

[1370]	val-error:0.04113
[1371]	val-error:0.04113
[1372]	val-error:0.04113
[1373]	val-error:0.04113
[1374]	val-error:0.04113
[1375]	val-error:0.04113
[1376]	val-error:0.04113
[1377]	val-error:0.04113
[1378]	val-error:0.04113
[1379]	val-error:0.04113
[1380]	val-error:0.04113
[1381]	val-error:0.04113
[1382]	val-error:0.04113
[1383]	val-error:0.04113
[1384]	val-error:0.04113
[1385]	val-error:0.04113
[1386]	val-error:0.04113
[1387]	val-error:0.04113
[1388]	val-error:0.04113
[1389]	val-error:0.04113
[1390]	val-error:0.04113
[1391]	val-error:0.04113
[1392]	val-error:0.04113
[1393]	val-error:0.04113
[1394]	val-error:0.04113
[1395]	val-error:0.04113
[1396]	val-error:0.04113
[1397]	val-error:0.04113
[1398]	val-error:0.04113
[1399]	val-error:0.04113
[1400]	val-error:0.04113
[1401]	val-error:0.04113
[1402]	val-error:0.04113
[1403]	val-error:0.04113
[1404]	val-error:0.04113
[1405]	val-error:0.041255
[1406]	val-error:0.041255
[1407]	val-error:0.04113
[1408]	val-error:0.04113
[1409]	val-error:0.0411

[1692]	val-error:0.04088
[1693]	val-error:0.04088
[1694]	val-error:0.04088
[1695]	val-error:0.04088
[1696]	val-error:0.04088
[1697]	val-error:0.04088
[1698]	val-error:0.04088
[1699]	val-error:0.041005
[1700]	val-error:0.04088
[1701]	val-error:0.04088
[1702]	val-error:0.04088
[1703]	val-error:0.04088
[1704]	val-error:0.04088
[1705]	val-error:0.04088
[1706]	val-error:0.04088
[1707]	val-error:0.04088
[1708]	val-error:0.04088
[1709]	val-error:0.04088
[1710]	val-error:0.041005
[1711]	val-error:0.041005
[1712]	val-error:0.041005
[1713]	val-error:0.041005
[1714]	val-error:0.041005
[1715]	val-error:0.041005
[1716]	val-error:0.041005
[1717]	val-error:0.041005
[1718]	val-error:0.041005
[1719]	val-error:0.041005
[1720]	val-error:0.041005
[1721]	val-error:0.04088
[1722]	val-error:0.04088
[1723]	val-error:0.04088
[1724]	val-error:0.04088
[1725]	val-error:0.04088
[1726]	val-error:0.04088
[1727]	val-error:0.04088
[1728]	val-error:0.04088
[1729]	val-error:0.04088
[1730]	val-error:0.04088
[1731]	val-er

[2015]	val-error:0.04088
[2016]	val-error:0.04088
[2017]	val-error:0.04088
[2018]	val-error:0.04088
[2019]	val-error:0.04088
[2020]	val-error:0.041005
[2021]	val-error:0.041005
[2022]	val-error:0.041005
[2023]	val-error:0.041005
[2024]	val-error:0.041005
[2025]	val-error:0.041005
[2026]	val-error:0.041005
[2027]	val-error:0.041005
[2028]	val-error:0.041005
[2029]	val-error:0.041005
[2030]	val-error:0.041005
[2031]	val-error:0.041005
[2032]	val-error:0.041005
[2033]	val-error:0.041005
[2034]	val-error:0.041005
[2035]	val-error:0.04113
[2036]	val-error:0.04113
[2037]	val-error:0.04113
[2038]	val-error:0.04113
[2039]	val-error:0.04113
[2040]	val-error:0.041005
[2041]	val-error:0.041005
[2042]	val-error:0.04113
[2043]	val-error:0.04113
[2044]	val-error:0.041255
[2045]	val-error:0.041255
[2046]	val-error:0.041255
[2047]	val-error:0.041255
[2048]	val-error:0.04113
[2049]	val-error:0.04113
[2050]	val-error:0.04113
[2051]	val-error:0.04113
[2052]	val-error:0.04113
[2053]	val-error:0.04113
[205

In [ ]:
0.47435618425825177
0.4803409503083061

In [ ]:
# feat_importance_table = pd.DataFrame(columns=['feat', 'importance'])
# feat_importance_table['feat'] = gbm.feature_name()
# feat_importance_table['importance'] = gbm.feature_importance()
feat_importance_table.sort_values('importance', ascending=False)

In [ ]:
# test_pre = gbm.predict(test_data, num_iteration=gbm.best_iteration)
submit['label'] = predictions_lgb / 5
submit.to_csv('../0111-03.csv', index=False)

In [ ]:
submit.head()